In [1]:
# Project #2, Abalone classification by sex
!pip install pycaret

In [2]:
# Import libraries
import pandas as pd
from pycaret.classification import *

In [3]:
# Load the dataset from UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
columns = [
    "Sex", "Length", "Diameter", "Height",
    "WholeWeight", "ShuckedWeight", "VisceraWeight", "ShellWeight", "Rings"
]
abalone = pd.read_csv(url, names=columns)

In [4]:
# Check data
abalone.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [5]:
# Check for class imbalance
abalone['Sex'].value_counts(normalize=True) * 100

,proportion
Sex,
M,36.581278
I,32.128322
F,31.290400


In [6]:
# Initialize PyCaret for classification
s = setup(
    data=abalone,
    target='Sex',
    session_id=123,
    fix_imbalance=True,     # handle class imbalance automatically
)

,Description,Value
0,Session id,123
1,Target,Sex
2,Target type,Multiclass
3,Target mapping,"F: 0, I: 1, M: 2"
4,Original data shape,"(4177, 9)"
5,Transformed data shape,"(4461, 9)"
6,Transformed train set shape,"(3207, 9)"
7,Transformed test set shape,"(1254, 9)"
8,Numeric features,8
9,Preprocess,True


In [7]:
# Compare models
best_model = compare_models(sort='F1') # Weighted F1. Reward balanced performance across all classes
print(best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.5416,0.7436,0.5416,0.5341,0.5361,0.3123,0.3133,0.4140
gbc,Gradient Boosting Classifier,0.5419,0.0000,0.5419,0.5322,0.5345,0.3134,0.3148,1.5780
lr,Logistic Regression,0.5501,0.0000,0.5501,0.5410,0.5331,0.3299,0.3372,0.9170
lda,Linear Discriminant Analysis,0.5440,0.0000,0.5440,0.5336,0.5323,0.3188,0.3227,0.0470
rf,Random Forest Classifier,0.5323,0.7385,0.5323,0.5277,0.5288,0.2983,0.2990,0.6820
xgboost,Extreme Gradient Boosting,0.5265,0.7260,0.5265,0.5227,0.5236,0.2896,0.2902,0.5170
lightgbm,Light Gradient Boosting Machine,0.5234,0.7337,0.5234,0.5178,0.5194,0.2849,0.2855,1.0680
knn,K Neighbors Classifier,0.5163,0.7021,0.5163,0.5171,0.5127,0.2769,0.2793,0.0720
ridge,Ridge Classifier,0.5385,0.0000,0.5385,0.5294,0.5124,0.3143,0.3263,0.0470
qda,Quadratic Discriminant Analysis,0.5299,0.0000,0.5299,0.5180,0.5047,0.2938,0.3042,0.0450


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)


In [8]:
# Tune the best model
tuned_model = tune_model(best_model, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4983,0.7541,0.4983,0.4738,0.4756,0.2531,0.2589
1,0.5529,0.7651,0.5529,0.5414,0.5448,0.3309,0.3325
2,0.4983,0.7519,0.4983,0.4835,0.4763,0.2548,0.2634
3,0.5342,0.7524,0.5342,0.5199,0.5087,0.3074,0.3177
4,0.5034,0.7502,0.5034,0.4890,0.4857,0.2604,0.2659
5,0.5274,0.7423,0.5274,0.5213,0.5145,0.2959,0.3013
6,0.5788,0.7600,0.5788,0.5708,0.5699,0.3708,0.3741
7,0.5514,0.7570,0.5514,0.5396,0.5329,0.3319,0.3394
8,0.5411,0.7490,0.5411,0.5321,0.5317,0.3143,0.3170


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [9]:
# Stack top models
top_models = compare_models(n_select=3, sort='F1')  # pick top 3

for i in range(len(top_models)):
    print(f"Top {i+1}: {top_models[i]}")

stacked_model = stack_models(top_models, optimize='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.5416,0.7436,0.5416,0.5341,0.5361,0.3123,0.3133,0.5200
gbc,Gradient Boosting Classifier,0.5419,0.0000,0.5419,0.5322,0.5345,0.3134,0.3148,1.5710
lr,Logistic Regression,0.5501,0.0000,0.5501,0.5410,0.5331,0.3299,0.3372,0.1220
lda,Linear Discriminant Analysis,0.5440,0.0000,0.5440,0.5336,0.5323,0.3188,0.3227,0.0500
rf,Random Forest Classifier,0.5323,0.7385,0.5323,0.5277,0.5288,0.2983,0.2990,0.7080
xgboost,Extreme Gradient Boosting,0.5265,0.7260,0.5265,0.5227,0.5236,0.2896,0.2902,0.2940
lightgbm,Light Gradient Boosting Machine,0.5234,0.7337,0.5234,0.5178,0.5194,0.2849,0.2855,0.8530
knn,K Neighbors Classifier,0.5163,0.7021,0.5163,0.5171,0.5127,0.2769,0.2793,0.0750
ridge,Ridge Classifier,0.5385,0.0000,0.5385,0.5294,0.5124,0.3143,0.3263,0.0450
qda,Quadratic Discriminant Analysis,0.5299,0.0000,0.5299,0.5180,0.5047,0.2938,0.3042,0.0460


Processing:   0%|          | 0/67 [00:00<?, ?it/s]

Top 1: ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)
Top 2: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                    

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5700,0.0000,0.5700,0.5692,0.5695,0.3531,0.3531
1,0.5392,0.0000,0.5392,0.5461,0.5423,0.3065,0.3067
2,0.5597,0.0000,0.5597,0.5627,0.5610,0.3386,0.3387
3,0.5514,0.0000,0.5514,0.5417,0.5443,0.3277,0.3290
4,0.5274,0.0000,0.5274,0.5223,0.5244,0.2905,0.2907
5,0.5240,0.0000,0.5240,0.5185,0.5182,0.2883,0.2903
6,0.5685,0.0000,0.5685,0.5640,0.5661,0.3517,0.3518
7,0.5514,0.0000,0.5514,0.5397,0.5430,0.3255,0.3269
8,0.5479,0.0000,0.5479,0.5450,0.5464,0.3207,0.3208


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
# Evaluate the final stacked model
evaluate_model(stacked_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [11]:
# Predict on new data
predictions = predict_model(stacked_model, data=abalone)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.8602,0.9696,0.8602,0.8596,0.8599,0.7898,0.7898


In [12]:
# Inspect top rows of the predictions DataFrame
predictions.head()

,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings,Sex,prediction_label,prediction_score
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,M,M,0.7520
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,M,M,0.5823
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,F,F,0.9097
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,M,M,0.7806
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,I,I,0.9399


In [13]:
# Save the best trained model pipeline to a file
save_model(best_model, 'abalone_sex_classifier_pipeline')

# Load the saved model pipeline from file
loaded_model = load_model('abalone_sex_classifier_pipeline')

# Display the loaded model object
print("Loaded model pipeline:")
print(loaded_model)

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded
Loaded model pipeline:
Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Length', 'Diameter', 'Height',
                                             'WholeWeight', 'ShuckedWeight',
                                             'VisceraWeight', 'ShellWeight',
                                             'Rings'],
                                    transformer=SimpleImputer(add_i...
                 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                      class_weight=None, criterion='gini',
                                      max_d